# Import 

In [5]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups


from keras.models import Sequential

from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.layers import Dense, Activation


from keras.utils.data_utils import get_file

from keras.callbacks import LambdaCallback

import gensim
import string


# Loading dataset

In [76]:
maxLen_sentence = 40


categories = ['rec.autos','rec.sport.hockey','comp.graphics','sci.space']    
dataset = fetch_20newsgroups(shuffle=True, random_state=0, categories=categories, remove=('headers','footers','quotes'))
train_corpus = dataset.data  

## Loding word2vec model and creating word vector representation of documents.

In [17]:
sentences = [[word for word in doc.lower().translate(str.maketrans('','',string.punctuation)).split()[:maxLen_sentence]] for doc in train_corpus]
word_model = gensim.models.Word2Vec(sentences, size=100, min_count=1, window=5, iter=100)
pretrained_weights = word_model.wv.syn0
vocab_size, emdedding_size = pretrained_weights.shape

C:\Users\mohit\Anaconda3\envs\py35\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  This is separate from the ipykernel package so we can avoid doing imports until


## Preaparing data for model training

In [18]:
train_x_sentences = np.zeros([len(sentences), maxLen_sentence], dtype=np.int32)
train_y_sentences = np.zeros([len(sentences)], dtype=np.int32)
for i, sentence in enumerate(sentences):
    if(len(sentence) != 0):
        for t, word in enumerate(sentence[:-1]):
            train_x_sentences[i, t] = word_model.wv.vocab[word].index
        train_y_sentences[i] = word_model.wv.vocab[sentence[-1]].index


## Creating sequentia
l model of word embedding and LSTM 

In [19]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=emdedding_size, weights=[pretrained_weights]))
model.add(LSTM(100))
model.add(Dense(units=vocab_size))
model.add(Activation('softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

## Fitting model with traing data

In [31]:
model.fit(train_x_sentences, train_y_sentences, batch_size=128, epochs=20)

Epoch 1/20
2371/2371 [==============================] - 22s 9ms/step - loss: 6.6101
Epoch 2/20
2371/2371 [==============================] - 22s 9ms/step - loss: 6.5220
Epoch 3/20
2371/2371 [==============================] - 23s 10ms/step - loss: 6.4918
Epoch 4/20
2371/2371 [==============================] - 22s 9ms/step - loss: 6.4719
Epoch 5/20
2371/2371 [==============================] - 22s 9ms/step - loss: 6.4563
Epoch 6/20
2371/2371 [==============================] - 22s 9ms/step - loss: 6.4456
Epoch 7/20
2371/2371 [==============================] - 22s 9ms/step - loss: 6.4267
Epoch 8/20
2371/2371 [==============================] - 22s 9ms/step - loss: 6.4131
Epoch 9/20
2371/2371 [==============================] - 23s 10ms/step - loss: 6.3939
Epoch 10/20
2371/2371 [==============================] - 23s 10ms/step - loss: 6.3790
Epoch 11/20
2371/2371 [==============================] - 23s 10ms/step - loss: 6.3558
Epoch 12/20
2371/2371 [==============================] - 22s 9ms/step 

## Predicting next word on test 'text' data - predict_post_processing

In [74]:
pre_text = 'A little more caution, and a little bit smarter in his own end'
for word in text.lower().split():
        if type(word_model.wv.vocab.get(word)) is not type(None):
            word_idxs = word_model.wv.vocab[word].index
            x=np.array(word_idxs)
            x.shape =  (1, )
            prediction = model.predict(x)
            print(prediction, len(prediction[0]))
            
            preds = np.asarray(prediction).astype('float64')
            preds = np.log(preds)
            exp_preds = np.exp(preds)
            preds = exp_preds / np.sum(exp_preds)
            probas = np.random.multinomial(1, preds.flatten(), 1)
            pred_idx = np.argmax(probas)

            pred_word = word_model.wv.index2word[pred_idx]
            print(word, word_idxs, pred_word, pred_idx)

#prediction = model.predict(x=np.array(word_idxs))

[[6.1669643e-04 4.8426763e-04 4.8619957e-04 ... 6.7238718e-05
  6.7682442e-05 6.7055356e-05]] 11574
a 1 locates 6502
[[1.2330088e-04 1.5929976e-04 1.7735061e-04 ... 8.1959130e-05
  8.3250525e-05 8.7663640e-05]] 11574
little 175 your 64
[[2.7769667e-04 3.3828052e-04 3.9021831e-04 ... 7.3552204e-05
  7.3666801e-05 7.4901342e-05]] 11574
more 72 varieties 6961
[[2.1878858e-04 2.3636142e-04 2.0960893e-04 ... 7.8828023e-05
  7.8989971e-05 7.8178113e-05]] 11574
and 5 walk 2094
[[6.1669643e-04 4.8426763e-04 4.8619957e-04 ... 6.7238718e-05
  6.7682442e-05 6.7055356e-05]] 11574
a 1 quality 1083
[[1.2330088e-04 1.5929976e-04 1.7735061e-04 ... 8.1959130e-05
  8.3250525e-05 8.7663640e-05]] 11574
little 175 manually 4944
[[1.0381615e-04 1.3200955e-04 1.5198936e-04 ... 8.3133440e-05
  8.4218744e-05 8.7206754e-05]] 11574
bit 233 mechanic 3807
[[1.5490265e-04 1.4213531e-04 1.4408700e-04 ... 8.2217135e-05
  8.3342507e-05 8.3257233e-05]] 11574
smarter 7612 televised 6119
[[3.1033049e-03 1.3283639e-03 1.3